## ETL

#### Importamos las librerias necesarias

In [126]:
# Importacion de librerias
import pandas as pd
import numpy as np
import json
import re
import ast
import warnings
warnings.filterwarnings('ignore')

#### Cargamos el archivo para la transformacion

In [127]:
# Carga del archivo .json
rows = []
with open("steam_games.json") as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

games = pd.DataFrame(rows)

In [128]:
# Visualizacion del contenido
games.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


In [129]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


#### Transformaciones

Desanidamos las columnas 'genres', 'tags' y 'specs'

In [130]:

# Función para eliminar corchetes y comillas simples de una cadena
def remove_brackets(text):
    text = text.replace("[", "").replace("]", "").replace("'", "")
    return text
 # Aplicar la función a las columnas deseadas
games['genres'] = games['genres'].astype(str).apply(remove_brackets)
games['tags'] = games['tags'].astype(str).apply(remove_brackets)
games['specs'] = games['specs'].astype(str).apply(remove_brackets)

Eliminamos las columnas que no se utilizara

In [131]:
# Eliminar las columnas 'url' y 'reviews_url'
column_drop_game = ['url', 'reviews_url']
games = games.drop(columns=column_drop_game)

#### Verificamos datos nulos en las columnas

In [132]:
#crear una funcion para los datos nulos
def evaluate_records(df):
	columns = df.columns
	for column in columns:
		if column != 'id': #Excluimos la columna 'id'
			null_count = df[column].isnull().sum()
			unique_count = len(df[column].unique())
			duplicate_count = df.duplicated(subset=column).sum()
			print(f"La Columna {column} tiene {null_count} registros nulos, un total de {unique_count} datos unicos, y {duplicate_count} registros duplicados entre ellos.")

In [133]:
# Rellenamos los valores nulos si se requiere
games['publisher'].fillna('NO', inplace=True)
games['genres'].fillna('NO', inplace=True)
games['app_name'].fillna('NO', inplace=True)
games['title'].fillna('NO', inplace=True)
games['release_date'].fillna(0, inplace=True)
games['tags'].fillna('NO', inplace=True)
games['discount_price'].fillna(0, inplace=True)
games['specs'].fillna('NO', inplace=True)
games['price'].fillna(0, inplace=True)
games['early_access'].fillna('NO', inplace=True)
games['developer'].fillna('NO', inplace=True)
games['sentiment'].fillna('NO', inplace=True)
games['metascore'].fillna('NO', inplace=True)

In [134]:
# Revisamos todo el contenido si no quedaron nulos
evaluate_records(games)

La Columna publisher tiene 0 registros nulos, un total de 8240 datos unicos, y 23895 registros duplicados entre ellos.
La Columna genres tiene 0 registros nulos, un total de 884 datos unicos, y 31251 registros duplicados entre ellos.
La Columna app_name tiene 0 registros nulos, un total de 32095 datos unicos, y 40 registros duplicados entre ellos.
La Columna title tiene 0 registros nulos, un total de 30055 datos unicos, y 2080 registros duplicados entre ellos.
La Columna release_date tiene 0 registros nulos, un total de 3583 datos unicos, y 28552 registros duplicados entre ellos.
La Columna tags tiene 0 registros nulos, un total de 15396 datos unicos, y 16739 registros duplicados entre ellos.
La Columna discount_price tiene 0 registros nulos, un total de 83 datos unicos, y 32052 registros duplicados entre ellos.
La Columna specs tiene 0 registros nulos, un total de 4650 datos unicos, y 27485 registros duplicados entre ellos.
La Columna price tiene 0 registros nulos, un total de 163 dat

In [135]:
games.shape

(32135, 14)

Cambiamos de formato la Fecha AAAA-mm-dd y creamos una nueva columna 'release_year'

In [136]:
# Cambio el formato de Fecha
games['release_date'] = pd.to_datetime(games['release_date'], format='%Y-%m-%d', errors='coerce')
games['release_year'] = games['release_date'].dt.year

In [137]:
# Vemos como queda
games[['release_date','release_year']].head()

,release_date,release_year
0,2018-01-04,2018.0
1,2018-01-04,2018.0
2,2017-07-24,2017.0
3,2017-12-07,2017.0
4,NaT,NaN


In [138]:
# Convertivos la columna release_year a tipo float
games['release_year'] = games['release_year'].astype(np.float64)

In [139]:
# Llenamos los nulos de la columna 'release_year' con 0
games['release_year'].fillna(0, inplace=True)

In [140]:
# Vemos como queda
games[['release_year']].head()

,release_year
0,2018.0
1,2018.0
2,2017.0
3,2017.0
4,0.0


In [141]:
# Extraer el texto utilizando expresiones regulares
games['free to play'] = games['price'].str.extract(r'([A-Za-z\s]+)')

# Rellenamos nulos con NO
games['free to play'].fillna('NO', inplace=True)


# Extraer los números utilizando una expresión regular más amplia
games['real_price'] = games['price'].apply(lambda x: re.findall(r'\d+\.\d+|\d+', str(x)))

# Convertir la columna a tipo float
games['real_price'] = games['real_price'].apply(lambda x: float(x[0]) if len(x) > 0 else None)

# Rellenamos nulos con 0
games['real_price'].fillna(0, inplace=True)

In [143]:
# Imprmir las columnas cradas para ver como quedo
games[['free to play','real_price']].head()

,free to play,real_price
0,NO,4.99
1,Free To Play,0.00
2,Free to Play,0.00
3,NO,0.99
4,NO,2.99


In [142]:
games.head()

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore,release_year,free to play,real_price
0,Kotoshiro,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"Strategy, Action, Indie, Casual, Simulation",4.49,Single-player,4.99,False,761140,Kotoshiro,NO,NO,2018.0,NO,4.99
1,"Making Fun, Inc.","Free to Play, Indie, RPG, Strategy",Ironbound,Ironbound,2018-01-04,"Free to Play, Strategy, Indie, RPG, Card Game,...",0.00,"Single-player, Multi-player, Online Multi-Play...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NO,2018.0,Free To Play,0.00
2,Poolians.com,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"Free to Play, Simulation, Sports, Casual, Indi...",0.00,"Single-player, Multi-player, Online Multi-Play...",Free to Play,False,670290,Poolians.com,Mostly Positive,NO,2017.0,Free to Play,0.00
3,彼岸领域,"Action, Adventure, Casual",弹炸人2222,弹炸人2222,2017-12-07,"Action, Adventure, Casual",0.83,Single-player,0.99,False,767400,彼岸领域,NO,NO,2017.0,NO,0.99
4,NO,nan,Log Challenge,NO,NaT,"Action, Indie, Casual, Sports",1.79,"Single-player, Full controller support, HTC Vi...",2.99,False,773570,NO,NO,NO,0.0,NO,2.99


In [144]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   publisher       32135 non-null  object        
 1   genres          32135 non-null  object        
 2   app_name        32135 non-null  object        
 3   title           32135 non-null  object        
 4   release_date    29783 non-null  datetime64[ns]
 5   tags            32135 non-null  object        
 6   discount_price  32135 non-null  float64       
 7   specs           32135 non-null  object        
 8   price           32135 non-null  object        
 9   early_access    32135 non-null  bool          
 10  id              32133 non-null  object        
 11  developer       32135 non-null  object        
 12  sentiment       32135 non-null  object        
 13  metascore       32135 non-null  object        
 14  release_year    32135 non-null  float64       
 15  fr

Optimizacion de datos

In [146]:
# Seleccionar las columnas deseadas
games = games[['id', 'publisher', 'genres', 'app_name', 'tags', 
                               'specs', 'early_access',  'developer', 
                               'sentiment', 'metascore', 'release_year', 'free to play', 'real_price']]

In [147]:
# Guardar el DataFrame como un archivo CSV
games.to_csv('C:/Users/JeshudjLenovo/Desktop/PI MLOps - STEAM/clean_games.csv', index=False, header=True, sep=';', encoding='utf-8')

In [148]:
# Número de observaciones en el DataFrame
num_observaciones = len(games)
print(f"Número de observaciones: {num_observaciones}")

Número de observaciones: 32135
